In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('tiling_results.parquet')
df.shape

(900, 15)

In [3]:
len(df[df.tiling_compatible == True]) 

110

In [4]:
counts = df['incompatible_reason'].value_counts(dropna=False).reset_index()
counts.columns = ['Value', 'Count']
print(counts)

                    Value  Count
0      unsupported_format    227
1          cant_open_file    206
2  cant_extract_variables    195
3  tile_generation_failed    134
4                    None    110
5                 timeout     18
6       failed_to_extract     10


In [6]:
df[df.incompatible_reason == 'cant_open_file']

,collection_concept_id,concept_id,data_center,data_url,backend,format,extension,tiling_compatible,incompatible_reason,error_message,tiles_url,variable,data_variables,num_granules,processing_level
12,C1593392869-LAADS,G2101669560-LAADS,UWI-MAD/SSEC,s3://prod-lads/CLDMSK_L2_MODIS_Aqua/CLDMSK_L2_...,None,NetCDF-4,nc,False,cant_open_file,Error opening s3://prod-lads/CLDMSK_L2_MODIS_A...,None,None,None,2437598.0,2
15,C2545314550-LPCLOUD,G2723304721-LPCLOUD,NASA/GSFC/SED/ESD/TISL/LandSIPS,s3://lp-prod-protected/VNP21.002/VNP21.A201602...,None,netCDF-4,nc,False,cant_open_file,Error opening s3://lp-prod-protected/VNP21.002...,None,None,None,1198247.0,2
20,C1562021084-LAADS,G2764618005-LAADS,NASA/GSFC/SED/ESD/HBSL/BISB/MODAPS,s3://prod-lads/CLDMSK_L2_VIIRS_SNPP/CLDMSK_L2_...,None,NetCDF-4,nc,False,cant_open_file,Error opening s3://prod-lads/CLDMSK_L2_VIIRS_S...,None,None,None,1185272.0,2
26,C2545310947-LPCLOUD,G2724728978-LPCLOUD,NASA/GSFC/SED/ESD/TISL/LandSIPS,s3://lp-prod-protected/VNP03MODLL.002/VNP03MOD...,None,HDF-EOS5,h5,False,cant_open_file,Error opening s3://lp-prod-protected/VNP03MODL...,None,None,None,1195367.0,1A
30,C1964798938-LAADS,G2533565274-LAADS,NASA/GSFC/SED/ESD/HBSL/BISB/MODAPS,s3://prod-lads/CLDMSK_L2_VIIRS_NOAA20/CLDMSK_L...,None,NetCDF-4,nc,False,cant_open_file,Error opening s3://prod-lads/CLDMSK_L2_VIIRS_N...,None,None,None,676219.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,C3555841897-OB_CLOUD,G3695454622-OB_CLOUD,OBPG,s3://ob-cumulus-prod-public/PACE_HARP2.2025090...,None,netCDF-4,nc,False,cant_open_file,Error opening s3://ob-cumulus-prod-public/PACE...,None,None,None,84584.0,1
877,C2623694321-GES_DISC,G2623696434-GES_DISC,NASA/GSFC/SED/ESD/TISL/GESDISC,s3://gesdisc-cumulus-prod-protected/HAQAST/HAQ...,None,netCDF,nc,False,cant_open_file,Error opening s3://gesdisc-cumulus-prod-protec...,None,None,None,1101.0,4
881,C3630967661-NSIDC_CPRD,G3813498543-NSIDC_CPRD,NASA NSIDC DAAC,s3://nsidc-cumulus-prod-protected/SMAP/SPL2SMP...,None,None,h5,False,cant_open_file,Error opening s3://nsidc-cumulus-prod-protecte...,None,None,None,1179.0,2
893,C3406446207-OB_CLOUD,G3547788309-OB_CLOUD,OBPG,s3://ob-cumulus-prod-public/S3A_OLCI_EFRNT.202...,None,netCDF-4,nc,False,cant_open_file,Error opening s3://ob-cumulus-prod-public/S3A_...,None,None,None,52166.0,2


In [33]:
# Filter out None/NaN and empty arrays
subset_df = df[df['data_variables'].apply(
    lambda x: x is not None and (isinstance(x, np.ndarray) and (x.size > 0 and x[0] != None))
)]
subset_df.data_variables

27                                               [SWIR1]
29                                               [TIRS2]
34     [latitude, longitude, scan_datetime, year, mon...
37     [aerosol_optical_thickness_550_land, aerosol_o...
43     [Aerosol_Optical_Thickness_550_Expected_Uncert...
                             ...                        
623    [Delta_Time, SZA, VZA, SAz, VAz, Longitude, La...
625                                             [weight]
640    [obs_id, fov_obs_id, obs_time_tai93, obs_time_...
643    [EFLUXICE, EFLUXWTR, FRSEAICE, HFLUXICE, HFLUX...
646                    [OriginalInputFiles, XmlMetadata]
Name: data_variables, Length: 160, dtype: object

In [47]:
all_unique_vars = set().union(*subset_df.data_variables)

keywords = ['time', 'latitude', 'longitude', 'num_obs', 'foo', 'crs']
filtered_list = [item for item in all_unique_vars if not any(keyword in item.lower() for keyword in keywords)]
print(len(filtered_list))

509


In [36]:
with open('titiler_cmr_compatibility/known_vars.txt', 'w') as f:
    for item in filtered_list:
        f.write(f"{item}\n")

In [11]:
# !aws s3 cp s3://nasa-veda-scratch/aimee/tiling_results.parquet \
#     s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet

copy: s3://nasa-veda-scratch/aimee/tiling_results.parquet to s3://nasa-veda-scratch/aimee/tiling_results_11-08-2025.parquet
